In [33]:
import pandas as pd
import geopandas as gp
import json

import warnings
warnings.filterwarnings("ignore")

In [34]:
def area_acres(df_merged):
    
    """
    function that takes in a dataframe as input and adds column of area acre in it
    """
    
    df_merged['Area Acres'] =  (df_merged.area / 4046.8564224).round(2)

In [35]:
def area_hectares(df_merged):
    
    """
    function that takes in a dataframe as input and adds column of area hectare in it
    """
    
    df_merged['Area Hectares'] =  (df_merged.area / 10000).round(2)

In [36]:
def crop_dic_creator():
    
    """
    function that creates a dictonary with 'Crop ID' as keys and 'Crop Name' as values
    """
    
    crop_names = ['urban','sugarcane','cotton','maize','others','orchards','juaar','rice','other vegetation','chilli','mustard','canola','banana','tobacco','wheat','tomato','mountain','esurvey','CP-77400',
                    'CPF-237',
                    'CPF-246',
                    'CPF-253',
                    'HSF-240',
                    'HSF-242',
                    'NSG-59',
                    'Other Variety',
                    'Ponda',
                    'SPF-234'
]
    
    
    
    crop_types_dic = dict(zip(range(len(crop_names)),crop_names))
    
    crop_types_dic[100] = 'esurvey'
    
    # Stress 
    crop_types_dic[77] = 'Stress'
    crop_types_dic[78] = 'Potential Stress'
    crop_types_dic[79] = 'No Stress'
    crop_types_dic[80] = 'High Vigour'
    
    
    
    # Health
    
    crop_types_dic[73] = 'Low Vegetation'
    crop_types_dic[74] = 'Moderate'
    crop_types_dic[75] = 'Good'
    crop_types_dic[76] = 'Excellent'
    

    return crop_types_dic



In [37]:
def questions():

    """
    Asking file no's that need to merged ans storing it in variable "files"
    
    """
    
    global l_type
    global area
    global paths
    
    paths = []
    
    
    print("Type '1' for L1 or '2' for L3 : ")
    l_type = int(input())
    
    
    print('How many Files are there ?')
    files = int(input())
    
    # Asking path of each file and storing in array "paths"

    for i in range(files):

        print('Enter Path of File No ', i + 1)
        path = input()    
        path = path.strip('"')
        paths.append(path)
        
          
    # Asking  Unit of Area and storing it in variable "area"
    print("Type '1' for Area in Acres or '2' for Area in Hectares : ")
    area = input()


In [63]:
def report_tileset_creation():
   

    # Reading all dataframes from paths to array "dfs" 
    
    global dfs
    dfs = []

    for i in paths:
        dfs.append(gp.read_file(i))


    # combining all dataframes from array "dfs" to make one dataframe "df_merged"

    df_merged = pd.concat(dfs, ignore_index=True)

    # storing orignal crs of dataframe

    og_crs = df_merged.crs
    
    # Changing the crs of dataframe to estimate crs

    estimated_utm_crs = df_merged.estimate_utm_crs().to_string()

    df_merged = df_merged.to_crs(estimated_utm_crs)

    # changing datatype of column 'predicted' to integer

    df_merged['predicted'] = df_merged['predicted'].astype(int)

    # changing name of column 'predicted' to 'crop id'

    df_merged = df_merged.rename(columns = {'predicted': 'c_id'})
    

    # making variable called 'pred_arr' that stores all unique crop id in data

    pred_arr = df_merged['c_id'].unique()
    
    # Splitting merged_df to different dataframes that has seperate crops and storing them in array "dfs_out"
    
    # If Mapbox is for L1 
    
    if l_type == '1': 
        
        dfs_out = []

        for i in pred_arr:
            dfs_out.append(df_merged[df_merged['c_id'] == i]) 
        
    
        # For each dataframe in dfs_out
        for i in range(len(dfs_out)):

            # Adding area column

            if area == '1':
                area_acres(dfs_out[i])
            elif area == '2':
                area_hectares(dfs_out[i])

            # Changing crs back to orignal crs    

            dfs_out[i] = dfs_out[i].to_crs(og_crs)

            # reseting index of all dataframes

            dfs_out[i] = dfs_out[i].reset_index(drop=True)

            # incremental id

            dfs_out[i]['id'] = dfs_out[i].index+1

            # Saving the file as geojson with crop name as geojson

            cid = dfs_out[i].iloc[0]['c_id']

            filename = crop_id_to_name_dic[cid]

            dfs_out[i] = dfs_out[i].drop(['c_id'], axis=1)

            temp_path = filename +'.geojson'

            dfs_out[i].to_file(temp_path, driver='GeoJSON')
            
        return dfs_out
    
    else:
        
            # Incremental ID

            df_merged['id'] = df_merged.index + 1
            
            # Adding Area Column
            
            if area == '1':
                area_acres(df_merged)
            elif area == '2':
                area_hectares(df_merged)
            
            # Adding Variety Column
            
            df_merged['Variety'] = df_merged['c_id'].apply(lambda x : crop_id_to_name_dic[x])
            
            df_merged.to_file('L3_mapbox.geojson', driver='GeoJSON')
            
            return df_merged

In [50]:
questions()

Type '1' for L1 or '2' for L3 : 
2
How many Files are there ?
1
Enter Path of File No  1
"C:\Users\FARMDAR\Desktop\Farmdar\Dashboard Automation\Data\Mehran\Dummy\model.geojson"
Type '1' for Area in Acres or '2' for Area in Hectares : 
1


In [64]:
crop_id_to_name_dic = crop_dic_creator()
dfs_out = report_tileset_creation()

In [44]:
dfs_out